In [1]:
# Package installations to work on WIRE

! pip install mamba
! mamba install gensim openai -y
! mamba install -c anaconda nltk -y
! mamba install -c conda-forge spacy -y
! mamba install -c conda-forge pyldavis -y
! mamba install -c conda-forge pypdf2 -y
! pip install transformers sentence-transformers
! pip install scikit-network
! pip install git+https://github.com/rwalk/gsdmm.git
! pip install gensim

! python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: mamba: command not found

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-ezifjbpr
  Running command git clone --filter=blob:none --quiet https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-ezifjbpr
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To updat

In [5]:
# imports

import csv
import json
import os
import re
from pprint import pprint
import time
import re
import unicodedata
import os
import sys
from contextlib import contextmanager
from copy import deepcopy
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import SpectralEmbedding
import pickle
#import sknetwork as skn
from random import *

import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

import nltk
nltk.download(["names", "stopwords", "state_union", "twitter_samples", "movie_reviews", "averaged_perceptron_tagger", "vader_lexicon", "punkt", "wordnet"])
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
ps = nltk.porter.PorterStemmer()


from sentence_transformers import SentenceTransformer

import spacy
#nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

from itertools import combinations


from collections import Counter
# set seed for reproducibility
# np.random.seed(493)

[nltk_data] Downloading package names to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to C:\Users\Seth
[nltk_data]     Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Seth Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package aver

In [6]:
import os, re, numpy as np, pandas as pd
from tqdm import tqdm
from collections import Counter
from scipy import stats
from hdbscan import HDBSCAN
import umap
import openai
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import LangChain
from bertopic.representation import TextGeneration

from sentence_transformers  import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

from googleapiclient.discovery import build

from langchain import OpenAI, PromptTemplate, LLMChain, HuggingFacePipeline, FewShotPromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.docstore.document import Document
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

from matplotlib import pyplot as plt
import seaborn as sns

C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Seth Benson\AppData\Roaming\Python\Python311\site-pac

## Prepare LLM Model

In [7]:
# Specify the LLM to use, Langchain verison

llm = HuggingFacePipeline.from_model_id(model_id="declare-lab/flan-alpaca-gpt4-xl", task = 'text2text-generation', device=-1,
                                      model_kwargs={"max_length":500, "no_repeat_ngram_size":2})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [48]:
# Create a generic prompt template

template = '''
The following is a statement from a cable news transcript. Please classify whether the statement is POSITIVE, NEUTRAL, or NEGATIVE towards "{subject}". Only return the stance.

statement: {statement}
'''

stance_prompt = PromptTemplate(
    input_variables=["subject","statement"],
    template=template
)

## Organizing Transcript-Topics

In [51]:
def transcript_cleaning(month):
    #import spacy stuff
    sid = SentimentIntensityAnalyzer()
    nlp = spacy.load("en_core_web_sm")
    NER = spacy.load("en_core_web_sm")

    #set up llm df
    llm_df = pd.DataFrame(columns = ['subject','sentence','output'])

    tr_tp_st = [[], [], [], []]

    gl = pd.read_csv("foxGuestList.csv", encoding="windows-1252")
    fox = open("fox_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Sentence'] = lines['Line'].apply(lambda x: [str(sent) for sent in nlp(x).sents])
            sentences = lines.explode('Sentence')
            sentences = sentences.dropna(subset = ['Sentence'])

            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                #excluding unrelevant entities
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            words = words[0:5]
            for w in words:
                    #try:
                        w1 = w[0]
                        wdf = sentences[sentences['Sentence'].astype(str).str.contains(str(w1))]
                        wdf = wdf.reset_index()
                        if len(wdf > 2): #need at least three sentences to be added as an entity
                            w_list.append(w1)
                            sent_list = []
                            #taking average (llm determined) sentiment from up to ten sentences with each entity
                            for i, r in wdf.iterrows():
                                if i < 10 and len(r['Line']) > 10:
                                    print(r['Sentence'])
                                    print(w1)
                                    stance = llm(stance_prompt.format(subject = w1, statement=r['Line']))
                                    d = {'NEGATIVE': -1, 'NEUTRAL': 0, 'POSITIVE': 1}
                                    stance1 = d[stance]
                                    sent_list.append(stance1)
                                    print(stance)
                                    print(stance1)
                                    llm_df.loc[len(llm_df)] = [w1,r['Line'],stance]
                            
                            word_sentiment_list.append(sent_list)
                    #except:
                    #    print('error with the contains line')

            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])

    return tr_tp_st

In [53]:
# Save out processed and topic modeled data
months = ['July']#['January','February','March','April','May','June','July','August','September','October','November','December']
for m in months:
    tr_tp_st = transcript_cleaning(m)
    tdf = pd.DataFrame(zip(tr_tp_st[0], tr_tp_st[1], tr_tp_st[2], tr_tp_st[3]), 
                       columns=["Transcript", "Topic", "Sentiment", "Program"],)
    #with open('v9 processed_transcripts_' + m + '2020.pkl', 'wb') as file:
    #    pickle.dump(tr_tp_st, file)
    tdf.to_csv('processed_transcripts_llm_' + m + '2020.csv')

55
 We're going to look first live at Selma, Alabama, as we remember a civil rights icon, John Lewis, known as the conscience of Congress.
John Lewis
POSITIVE
1
Brown chapel AME church, John Lewis' family is beginning to arrive and those who will pay their respects today as well.
John Lewis
POSITIVE
1
 And from here we will go back, this is Selma, Alabama, on March 1st, as you see Congressman John Lewis there.
John Lewis
POSITIVE
1
Thank you for Congressman John Lewis.
John Lewis
POSITIVE
1
 So as they have wrapped up the very short ceremony at Brown Chapel AME church in Selma, Alabama, they're getting ready to do the ceremonial crossing with Congressman John Lewis carried by a caisson across that bridge.
John Lewis
POSITIVE
1
90
 We're going to look first live at Selma, Alabama, as we remember a civil rights icon, John Lewis, known as the conscience of Congress.
John
POSITIVE
1
Brown chapel AME church, John Lewis' family is beginning to arrive and those who will pay their respects tod

KeyboardInterrupt: 

## Alternative transcrip sorting (using nouns and lines)

In [42]:
def transcript_cleaning(month):
    #import spacy stuff
    sid = SentimentIntensityAnalyzer()
    #nlp = spacy.load("en_core_web_sm")
    NER = spacy.load("en_core_web_sm")

    tr_tp_st = [[], [], [], []]

    gl = pd.read_csv("foxGuestList.csv", encoding="windows-1252")
    fox = open("fox_text.txt", "r", encoding="windows-1252")
    fox = fox.readlines()
    for l in range(len(gl)):
        #######################################
        ##Turning transcript into df of lines##
        #######################################
        lines = pd.DataFrame(
            columns=["Date", "Start_Hour", "Program", "Title", "Speaker", "Line"]
        )
        # print(lines)
        transcript = fox[l]
        meta = gl.iloc[l, :]
        dt = meta["Date"]
        if month in str(dt):
            if randint(1, 100) <= 5:
                print(l)
            chunks = transcript.split("|")
            line = ""
            spkr = ""
            for c in chunks:
                if (
                    "Page" not in c[0:10]
                    and "...." not in c
                    and c[0:15] not in meta["Title"]
                ):
                    # print(c)
                    if ":" in c:
                        c = c.split(":")
                        if str.isupper(c[0]):
                            lines.loc[len(lines)] = [
                                meta["Date"],
                                meta["Start_Hour"],
                                meta["Program"],
                                meta["Title"],
                                spkr,
                                line,
                            ]
                            # print(spkr + ':' + line)
                            spkr = c[0]
                            line = c[1]
                    else:
                        line += c
            lines.loc[len(lines)] = [
                meta["Date"],
                meta["Start_Hour"],
                meta["Program"],
                meta["Title"],
                spkr,
                line,
            ]

            #########################
            ##Keyword and Sentiment##
            #########################

            lines['Line'] = lines['Line'].apply(lambda x: x.lower())
            #lines["VADER"] = lines["Line"].apply(
            #    lambda x: list(sid.polarity_scores(x).values())[3]
            #)

            #getting most used nouns
            """word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            tokens = nltk.word_tokenize(raw_text)
            words = Counter(tokens).most_common()
            words1 = []
            i = 0
            while len(words1) < 15 and i < len(words):
                w = words[i][0]
                #print(w)
                tag =  nltk.pos_tag([w])
                t = tag[0][1] 
                if t in ['NN','NNS','NNP','NNPS','FW','SYM'] and w not in ['video','clip','i','end','begin']: #or t == "FW":
                    words1.append(w)
                i += 1"""
            
            #getting most used known entities
            word_sentiment_list = []
            w_list = []
            raw_text= ' '.join(lines['Line'])
            text1 = NER(raw_text)
            words = []
            for word in text1.ents:
                if word.label_ not in ['TIME','MONEY','PERCENT','DATE','QUANTITY','ORDINAL','CARDINAL']:
                    words.append(word.text)
            words = Counter(words).most_common()
            words = words[0:15]

           #print(len(words))
            for w in words:
                #try:
                    w1 = w[0]
                    #print(w1)
                    wdf = lines[lines['Line'].astype(str).str.contains(str(w1))]
                    wdf = wdf.reset_index()
                    print(len(wdf))
                    w_list.append(w1)
                    sent_list = []
                    for i, r in wdf.iterrows():
                        if i < 5:
                            #r = wdf[i]
                            print(r['Line'])
                            print(w1)
                            stance = llm(stance_prompt.format(subject = w1, statement=r['Line']))
                            d = {'AGAINST': -1, 'NEUTRAL': 0, 'FOR': 1}
                            stance1 = d[stance]
                            sent_list.append(stance1)
                            print(stance)
                            print(stance1)
                    word_sentiment_list.append(sent_list)
                #except:
                #    pass

                
            ######################
            ##Storing Each Topic##
            ######################
            for w in range(len(w_list)):
                if len(word_sentiment_list[w]) > 2:
                    tr_tp_st[0].append(l)
                    tr_tp_st[1].append(w_list[w])
                    tr_tp_st[2].append(np.mean(word_sentiment_list[w]))
                    tr_tp_st[3].append(lines["Program"][0])


    return tr_tp_st